In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import optim

# For reproducability
torch.manual_seed(0)

# Full Description of the Convolutional Layer

In [2]:
layer = nn.Conv2d(in_channels = 3,
                  out_channels = 64,
                  kernel_size = (5, 5),
                  stride = 2,
                  padding = 1
                  )

# Closing the Loop on MNIST with Convolutional Networks

In [3]:
class MNISTConvNet(nn.Module):
  def __init__(self):
    super(MNISTConvNet, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 32, 5, padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 64, 5, padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.fc1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(7*7*64, 1024),
        nn.Dropout(0.5),
        nn.Linear(1024, 10)
    )

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    return self.fc1(x)

In [4]:
trainset = MNIST('.', train=True, download=True, 
                      transform=ToTensor())
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
lr = 1e-4
num_epochs = 40

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MNISTConvNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [7]:
for epochs in range(num_epochs):
  running_loss = 0.0
  num_correct = 0
  for inputs, labels in trainloader:
    optimizer.zero_grad()
    outputs = model(inputs.to(device))
    loss = loss_fn(outputs, labels.to(device))
    loss.backward()
    running_loss += loss.item()
    optimizer.step()
    _, idx = outputs.max(dim=1)
    num_correct += (idx == labels.to(device)).sum().item()
  print('Loss: {} Accuracy: {}', running_loss/len(trainloader),
        num_correct/len(trainloader))

Loss: {} Accuracy: {} 2.2972030723527044 8.143923240938166
Loss: {} Accuracy: {} 2.287285191163834 10.567164179104477
Loss: {} Accuracy: {} 2.2767548337419914 13.352878464818764
Loss: {} Accuracy: {} 2.2656721669727804 16.176972281449892
Loss: {} Accuracy: {} 2.2535015713177255 19.083155650319828
Loss: {} Accuracy: {} 2.2400468135439255 21.900852878464818
Loss: {} Accuracy: {} 2.224334346460127 24.52771855010661
Loss: {} Accuracy: {} 2.2063664607147673 26.96268656716418
Loss: {} Accuracy: {} 2.185721436289074 29.211087420042645
Loss: {} Accuracy: {} 2.161161944556084 31.571428571428573
Loss: {} Accuracy: {} 2.1313855142227367 33.785714285714285
Loss: {} Accuracy: {} 2.0958461542881883 35.38272921108742
Loss: {} Accuracy: {} 2.0525187367061055 37.195095948827294
Loss: {} Accuracy: {} 1.9985858819632134 38.68017057569296
Loss: {} Accuracy: {} 1.934484860536132 39.73880597014925
Loss: {} Accuracy: {} 1.8546446641879295 40.94456289978678
Loss: {} Accuracy: {} 1.7601065218829905 41.88805970

# Image Preprocessing Pipelines Enable More Robust Models

In [17]:
from torchvision import transforms

transform = transforms.Normalize(mean = (0.1307,),
                                 std = (0.3081,)
                                 )

In [18]:
transform = transforms.Compose([
      transforms.RandomCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ColorJitter(brightness=0,
                             contrast=0,
                             saturation=0,
                             hue=0),
      transforms.ToTensor(),
      transforms.Normalize(mean = (0.1307,),
                           std = (0.3081,)
                           )
      ])

# Accelerating Training with Batch Normalization


In [19]:
layer = nn.BatchNorm2d(num_features=32,
                       eps=1e-05,
                       momentum=0.1,
                       affine = True,
                       track_running_stats = True)

In [20]:
layer = nn.BatchNorm1d(num_features=32)

#Group normalization for memory constrained learning tasks


In [21]:
layer = nn.GroupNorm(num_groups=1,
                     num_channels=32)

#Building a Convolutional Network for CIFAR-10


In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
        )
        self.block2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128,10),
            nn.BatchNorm1d(10)
        )

    def forward(self, x):
        x = self.block1(x)
        return self.block2(x)

# Building a residual network with superhuman vision

In [23]:
from torchvision.models import resnet34

model = resnet34()

In [24]:
class ResidualBlock(nn.Module):
  def __init__(self, in_layers, out_layers, downsample=None):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_layers, out_layers,
                           kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(out_layers)
    self.conv2 = nn.Conv2d(out_layers, out_layers,
                           kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(out_layers)
    self.downsample = downsample
    self.relu = nn.ReLU(inplace=True)

  def forward(self, inp):
    # Residual block
    out = self.conv1(inp)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    
    if self.downsample:
      inp = self.downsample(inp)
    
    # Shortcut connection
    out += inp
    return out

In [25]:
downsample = nn.Sequential(
      nn.Conv2d(64, 128, kernel_size=1, stride=1, bias=False),
      nn.BatchNorm2d(128)
    )

In [26]:
class ResNet34(nn.Module):
  def __init__(self):
    super(ResNet34, self).__init__()

    self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=7,
                stride=2, padding=3, bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,
                   stride=2, padding=1)
    )

    # Note that each ResidualBlock has 2 conv layers
    # 3 blocks in a row, 6 conv layers
    self.comp1 = nn.Sequential(
      ResidualBlock(64, 64),
      ResidualBlock(64, 64),
      ResidualBlock(64, 64)
    )

    # 4 blocks in a row, 8 conv layers
    downsample1 = nn.Sequential(
      nn.Conv2d(64, 128, kernel_size=1,
             stride=1, bias=False),
      nn.BatchNorm2d(128)
    )
    self.comp2 = nn.Sequential(
      ResidualBlock(64, 128, downsample=downsample1),
      ResidualBlock(128, 128),
      ResidualBlock(128, 128),
      ResidualBlock(128, 128)
    )
    
    # 6 blocks in a row, 12 conv layers
    downsample2 = nn.Sequential(
      nn.Conv2d(128, 256, kernel_size=1, stride=1, bias=False),
      nn.BatchNorm2d(256)
    )
    self.comp3 = nn.Sequential(
      ResidualBlock(128, 256, downsample=downsample2),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
    )
    
    # 3 blocks in a row, 6 conv layers
    downsample3 = nn.Sequential(
      nn.Conv2d(256, 512, kernel_size=1, stride=1, bias=False),
      nn.BatchNorm2d(512)
    )
    self.comp4 = nn.Sequential(
      ResidualBlock(256, 512, downsample=downsample3),
      ResidualBlock(512, 512),
      ResidualBlock(512, 512)   
    )

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    # ImageNet classifier: 1000 classes
    self.fc = nn.Linear(512, 1000)

  def forward(self, inp):
    out = self.conv1(inp)
    
    out = self.comp1(out)
    out = self.comp2(out)
    out = self.comp3(out)
    out = self.comp4(out)

    out = self.avgpool(out)
    out = torch.flatten(out, 1)
    out = self.fc(out)

    return out

# ------------------

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

In [ ]:
# O'Reilly Code width is 64
################################################################

In [ ]:
#layer = nn.Conv2d(in_channels, out_channels, kernel_size)

#layer = nn.Conv2d(3, 64, (3,3), stride=2, padding=1)
layer = nn.Conv2d(3, 64, (3,3), stride=2, padding=1)
N, c, h, w = 4, 3, 32, 32

input = torch.randn(N, c, h, w)
output = layer(input)

print(m)
print(output.shape)

Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
torch.Size([4, 64, 16, 16])


In [ ]:
import torch.nn as nn

layer = nn.Conv2d(in_channels = 3,
                  out_channels = 64,
                  kernel_size = (5, 5),
                  stride = 2,
                  padding = 1
                  )
layer

Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))

Here, in_channels represents the depth or number of input planes. For color images, the number of input channels is often three representing the RGB channels. The nn.Conv2d layer will accept as an input a four-dimensional tensor of size N x Cin x hin x win, where N is the number of examples in our minibatch. The out_channels argument represents the number of output planes or feature maps. The kernel_size parameter determines the filter size while the stride and padding parameters determine the stride size and padding size respectfully. Note that equal dimension settings can be passed in with a single value as shown here with stride and padding.

p.122, MNIST

In [ ]:
class MNISTConvNet(nn.Module):
  def __init__(self):
    super(MNISTConvNet, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 32, 5, padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 64, 5, padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.fc1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(7*7*64, 1024),
        nn.Dropout(0.5),
        nn.Linear(1024, 10)
    )

  def forward(self, x):
    print(x.shape)
    x = self.conv1(x)
    print(x.shape)
    x = self.conv2(x)
    print(x.shape)
    x = self.fc1(x)
    print(x.shape)
    return x


In [ ]:
net = MNISTConvNet()
x = torch.rand((32,1,28,28))
y = net(x)
y[0]

torch.Size([32, 784])


RuntimeError: ignored

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

net = Net()
net

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.module1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
        )
        self.module2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128,10)
        )

    def forward(self, x):
        x = self.module1(x)
        x = self.module2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
net = Net()

In [ ]:
net

Net(
  (module1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
  )
  (module2): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=9216, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

# Image Preprocessing
p. 124

In [ ]:
from torchvision import transforms, datasets

In [ ]:
from torchvision import transforms

transform = transforms.Normalize(mean = (0.1307,),
                                 std = (0.3081,)
                                 )

In [ ]:
dataset = datasets.MNIST('.', download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
transform(dataset[0])

TypeError: ignored

In [ ]:
transform = transforms.Compose([
      transforms.RandomCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ColorJitter(brightness=0, 
                             contrast=0, 
                             saturation=0, 
                             hue=0),
      transforms.ToTensor(),
      transforms.Normalize(mean = (0.1307,),
                           std = (0.3081,)
                           )
      ])

# Batch Norm
p .126

In [ ]:
n_out = 32
nn.BatchNorm2d(n_out)

BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
import torch.nn as nn

layer = nn.BatchNorm2d(num_features=32,
                       eps=1e-05, 
                       momentum=0.1)

In [ ]:
for n in layer.named_parameters():
  print(n)

('weight', Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True))
('bias', Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True))


In [ ]:
def conv_batch_norm(x, n_out, phase_train):
  beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
  gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
  beta = tf.get_variable("beta", [n_out],  initializer=beta_init)
  gamma = tf.get_variable("gamma", [n_out],  initializer=gamma_init)
  batch_mean, batch_var = tf.nn.moments(x, [0,1,2],name='moments')

  ema = tf.train.ExponentialMovingAverage(decay=0.9)
  ema_apply_op = ema.apply([batch_mean, batch_var])
  ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)

def mean_var_with_update():
  with tf.control_dependencies([ema_apply_op]):
  return tf.identity(batch_mean),

  tf.identity(batch_var)
  mean, var = control_flow_ops.cond(phase_train,
  mean_var_with_update,
  lambda: (ema_mean, ema_var))
  normed = tf.nn.batch_norm_with_global_normalization(x,
  mean, var, beta, gamma, 1e-3, True)
  return normed

In [ ]:
import torch.nn.functional as F

In [ ]:
F.batch_norm()

TypeError: ignored

In [ ]:
layer = nn.BatchNorm1d(num_features=32,
                       eps=1e-05, 
                       momentum=0.1)

# CNN for CIFAR10
p.132

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.module1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
        )
        self.module2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128,10),
            nn.BatchNorm1d(10)
        )

    def forward(self, x):
        x = self.module1(x)
        x = self.module2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
net = Net()

In [ ]:
net

Net(
  (module1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.25, inplace=False)
  )
  (module2): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=9216, out_features=128, bias=True)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=128, out_features=10, bias=True)
    (6): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

# ResNets
# (Already in PyTorch)
p. 138
[tutorial](https://towardsdatascience.com/residual-network-implementing-resnet-a7da63c7b278)

In [ ]:
import torch.nn as nn

# Residual Block

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.activation = activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: 
          residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels


In [ ]:
# O'Reilly Code width is 64
################################################################

In [ ]:
class ResNet34(nn.Module):

	def __init__(self):
		super(ResNet34, self).__init__()

		self.conv1 = nn.Conv2d(3, 64, kernel_size=7, 
                         stride=2, padding=3, bias=False)
		self.bn1 = nn.BatchNorm2d(64)
		self.relu = nn.ReLU()
		self.maxpool = nn.MaxPool2d(kernel_size=3, 
                              stride=2, padding=1)

		# Note that each ResidualBlock has 2 conv layers
		# 3 blocks in a row, 6 conv layers
		self.comp1 = nn.Sequential(
			ResidualBlock(64, 64, kernel_size=3, 
                 stride=1, padding=1),
			ResidualBlock(64, 64, kernel_size=3, 
                 stride=1, padding=1),
			ResidualBlock(64, 64, kernel_size=3, 
                 stride=1, padding=1)
		)

		# 4 blocks in a row, 8 conv layers
		downsample1 = nn.Sequential(
			nn.Conv2d(64, 128, kernel_size=1, 
             stride=1, bias=False),
			nn.BatchNorm2d(128)
		)
		self.comp2 = nn.Sequential(
			ResidualBlock(64, 128, kernel_size=3, 
                 stride=1, padding=1, downsample=downsample1),
			ResidualBlock(128, 128, kernel_size=3, 
                 stride=1, padding=1),
			ResidualBlock(128, 128, kernel_size=3, 
                 stride=1, padding=1),
			ResidualBlock(128, 128, kernel_size=3, 
                 stride=1, padding=1)
		)
		
		# 6 blocks in a row, 12 conv layers
		downsample2 = nn.Sequential(
			nn.Conv2d(128, 256, kernel_size=1, stride=1, bias=False),
			nn.BatchNorm2d(256)
		)
		self.comp3 = nn.Sequential(
			ResidualBlock(128, 256, kernel_size=3, 
                 stride=1, padding=1, downsample=downsample2),
			ResidualBlock(256, 256, kernel_size=3, stride=1, padding=1),
			ResidualBlock(256, 256, kernel_size=3, stride=1, padding=1),
			ResidualBlock(256, 256, kernel_size=3, stride=1, padding=1),
			ResidualBlock(256, 256, kernel_size=3, stride=1, padding=1),
			ResidualBlock(256, 256, kernel_size=3, stride=1, padding=1),
		)
		
		# 3 blocks in a row, 6 conv layers
		downsample3 = nn.Sequential(
			nn.Conv2d(256, 512, kernel_size=1, stride=1, bias=False),
			nn.BatchNorm2d(512)
		)
		self.comp4 = nn.Sequential(
			ResidualBlock(256, 512, kernel_size=3, 
                 stride=1, padding=1, downsample=downsample3),
			ResidualBlock(512, 512, kernel_size=3, stride=1, padding=1),
			ResidualBlock(512, 512, kernel_size=3, stride=1, padding=1)		
		)

		self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
		# ImageNet classifier: 1000 classes
		self.fc = nn.Linear(512, 1000)

	def forward(self, inp):
		out = self.conv1(inp)
		out = self.bn1(out)
		out = self.relu(out)
		out = self.maxpool(out)

		out = self.comp1(out)
		out = self.comp2(out)
		out = self.comp3(out)
		out = self.comp4(out)

		out = self.avgpool(out)
		out = torch.flatten(out, 1)
		out = self.fc(out)

		return out

In [ ]:
model = ResNet34()

TypeError: ignored

In [ ]:
from torchvision import models

In [ ]:
models.resnet.BasicBlock(3,64)

BasicBlock(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
import torch.nn as nn

conv_block = nn.Sequential(
    nn.Conv2d(3,64,kernel_size=7, 
              stride=2, padding=3, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [ ]:
from torchvision.models.resnet import BasicBlock

residual_layer = BasicBlock(inplanes,
                            planes)

In [ ]:
from torch import nn

In [ ]:
class ResidualBlock(nn.Module):
    
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, 
                               kernel_size=3, 
                               stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, 
                               kernel_size=3, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample:
          identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

NameError: ignored

In [ ]:
from torchvision.models.resnet import BasicBlock, ResNet

In [ ]:
resnet34_layers = [3,4,6,3]
resnet34 = ResNet(block = BasicBlock,
                  layers = resnet34_layers)

In [ ]:
from torchvision.models import resnet34

In [ ]:
model = resnet34()

In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
class ResNet34(nn.Module):
  def __init__(self):
    super(ResNet34, self.__init__()
    
    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=7,
                  stride=2, padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLu(inplace=True),
        nnMaxPoll2d(kernel_size=2, stride=2, padding=1)
    )

    self.res_layer1 = nn.Sequential(
        ResidualBlock(64, 64)
    )


In [ ]:
downsample = nn.Sequential(
			nn.Conv2d(64, 128, kernel_size=1, stride=1, bias=False),
			nn.BatchNorm2d(128)
		)

NameError: ignored

# This one!


In [ ]:
from torch import nn

class ResidualBlock(nn.Module):
  def __init__(self, in_layers, out_layers, downsample=None):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_layers, out_layers,
                           kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(out_layers)
    self.conv2 = nn.Conv2d(out_layers, out_layers, 
                           kernel_size=3, stride=1, padding=1) 
    self.bn2 = nn.BatchNorm2d(out_layers)
    self.downsample = downsample
    self.relu = nn.ReLU(inplace=True)

  def forward(self, inp):
    # Residual block
    out = self.conv1(inp)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    
    if self.downsample:
      inp = self.downsample(inp)
    
    # Shortcut connection
    out += inp
    return out

In [ ]:
class ResNet34(nn.Module):
	def __init__(self):
		super(ResNet34, self).__init__()

		self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=7, 
                stride=2, padding=3, bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, 
                   stride=2, padding=1)
    )

		# Note that each ResidualBlock has 2 conv layers
		# 3 blocks in a row, 6 conv layers
		self.comp1 = nn.Sequential(
			ResidualBlock(64, 64),
			ResidualBlock(64, 64),
			ResidualBlock(64, 64)
		)

		# 4 blocks in a row, 8 conv layers
		downsample1 = nn.Sequential(
			nn.Conv2d(64, 128, kernel_size=1, 
             stride=1, bias=False),
			nn.BatchNorm2d(128)
		)
		self.comp2 = nn.Sequential(
			ResidualBlock(64, 128, downsample=downsample1),
			ResidualBlock(128, 128),
			ResidualBlock(128, 128),
			ResidualBlock(128, 128)
		)
		
		# 6 blocks in a row, 12 conv layers
		downsample2 = nn.Sequential(
			nn.Conv2d(128, 256, kernel_size=1, stride=1, bias=False),
			nn.BatchNorm2d(256)
		)
		self.comp3 = nn.Sequential(
			ResidualBlock(128, 256, downsample=downsample2),
			ResidualBlock(256, 256),
			ResidualBlock(256, 256),
			ResidualBlock(256, 256),
			ResidualBlock(256, 256),
			ResidualBlock(256, 256),
		)
		
		# 3 blocks in a row, 6 conv layers
		downsample3 = nn.Sequential(
			nn.Conv2d(256, 512, kernel_size=1, stride=1, bias=False),
			nn.BatchNorm2d(512)
		)
		self.comp4 = nn.Sequential(
			ResidualBlock(256, 512, downsample=downsample3),
			ResidualBlock(512, 512),
			ResidualBlock(512, 512)		
		)

		self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
		# ImageNet classifier: 1000 classes
		self.fc = nn.Linear(512, 1000)

	def forward(self, inp):
		out = self.conv1(inp)
		
		out = self.comp1(out)
		out = self.comp2(out)
		out = self.comp3(out)
		out = self.comp4(out)

		out = self.avgpool(out)
		out = torch.flatten(out, 1)
		out = self.fc(out)

		return out

In [ ]:
model = ResNet34()

In [ ]:
import torch 
x = torch.rand(32,3,224,224)
y = model(x)